In [2]:
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)
# response = llm.invoke("The first person to land on moon was ...")
# print(response.content)

In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.idp.com/job/Melbourne-Senior-Business-Analyst-VIC-3008/1062649666/")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.














Senior Business Analyst Job Details | idpeducati

























Press Tab to Move to Skip to Content Link
Skip to main content








View All Jobs
Expression of Interest




















Search by Keyword







                                 
                            





Clear












View All Jobs
Expression of Interest








Language 


English (United Kingdom)


한국어 (대한민국)


简体中文 (中国大陆)


繁體中文(台灣)





View profile




Employee Login






















Search by Keyword






Show More Options





Loading...







                                            Category
                                        


All





                                            Country
                                        


All









                                 
                            





Clear
















Select how often (in days) to receive an alert:



 Create Alert







×



Select how often (in days) to receive an alert:





In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)
print(res.content)

```json
{
  "role": "Senior Business Analyst",
  "experience": "Minimum 5+ years of work experience as a BA required in Software development projects",
  "skills": [
    "Business analysis",
    "Agile methodologies",
    "Digital transformation",
    "Multi-system integration",
    "Microsoft Office",
    "Confluence",
    "JIRA",
    "Strong interpersonal skills",
    "Strong written and oral communication skills"
  ],
  "description": "The Senior Business Analyst’s role will focus on supporting the discovery and analysis activity to deliver a digital customer experience that will assist in transforming IDP’s Student Placement business."
}
```


In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Business Analyst',
 'experience': 'Minimum 5+ years of work experience as a BA required in Software development projects',
 'skills': ['Business analysis',
  'Agile methodologies',
  'Digital transformation',
  'Multi-system integration',
  'Microsoft Office',
  'Confluence',
  'JIRA',
  'Strong interpersonal skills',
  'Strong written and oral communication skills'],
 'description': 'The Senior Business Analyst’s role will focus on supporting the discovery and analysis activity to deliver a digital customer experience that will assist in transforming IDP’s Student Placement business.'}

In [9]:
type(json_res)

dict

In [10]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [16]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
            metadatas={"links": row["Links"]},
            ids=[str(uuid.uuid4())])

In [17]:
job = json_res
job['skills']

['Business analysis',
 'Agile methodologies',
 'Digital transformation',
 'Multi-system integration',
 'Microsoft Office',
 'Confluence',
 'JIRA',
 'Strong interpersonal skills',
 'Strong written and oral communication skills']

In [19]:
job

{'role': 'Senior Business Analyst',
 'experience': 'Minimum 5+ years of work experience as a BA required in Software development projects',
 'skills': ['Business analysis',
  'Agile methodologies',
  'Digital transformation',
  'Multi-system integration',
  'Microsoft Office',
  'Confluence',
  'JIRA',
  'Strong interpersonal skills',
  'Strong written and oral communication skills'],
 'description': 'The Senior Business Analyst’s role will focus on supporting the discovery and analysis activity to deliver a digital customer experience that will assist in transforming IDP’s Student Placement business.'}

In [18]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfol

In [21]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Karthikeyan, a business development executive at KatQ. KatQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of KatQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase KatQ's portfolio: {link_list}
        Remember you are Karthikeyan, BDE at KatQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Business Analysis Solutions for IDP's Student Placement Business

Dear Hiring Manager,

I came across the job description for a Senior Business Analyst at IDP, and I am excited to introduce KatQ, an AI & Software Consulting company that can support your digital transformation journey. With our expertise in business process automation, we can help you deliver a seamless digital customer experience that transforms your Student Placement business.

Our team at KatQ has a proven track record of empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. We specialize in Agile methodologies, multi-system integration, and digital transformation, which align perfectly with the requirements of the Senior Business Analyst role.

I would like to highlight some of our notable projects that demonstrate our capabilities:

* Our expertise in digital transformation can be seen in our [Magento portfolio